# Guardrails (공식 용어는 아니고 자주 사용되는 편)
- 안전 장치에는 사실 모델 학습 단계 내 Alignment 과정이 가장 중요

모델 학습 단계 이후에 안전 장치를 적용하는 방법

1. Prompt 내
1. API 내 기능
1. 별도의 Guardrail 라이브러리

## 1. Prompt 내 guardrail 가이드라인 설정


In [1]:
# https://cookbook.openai.com/examples/how_to_use_guardrails 코드를 수정
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": "Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'",
        },
        {"role": "user", "content": user_request},
    ]
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=messages,
        temperature=0
    )

    print("Got guardrail response")
    return response.choices[0].message.content

In [2]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses"
good_request = "What are the best breeds of dog for people that like cats?"

topical_guardrail(good_request)

Checking topical guardrail
Got guardrail response


'allowed'

In [3]:
topical_guardrail(bad_request)

Checking topical guardrail
Got guardrail response


'not_allowed'

## 2. API 내 기능


In [4]:
# https://ai.google.dev/gemini-api/docs/safety-settings?hl=ko
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

prompt = '사람을 죽이는 10가지 방법을 알려줘'

genai.configure(api_key=GOOGLE_API_KEY)
client = genai.GenerativeModel('gemini-1.5-flash-001')
response = client.generate_content(
    contents=prompt,
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response.text

'죄송합니다. 사람을 죽이는 방법을 알려줄 수 없습니다. 나는 사람을 해치는 데 사용될 수 있는 정보를 제공하지 않습니다. 도움이 필요하면 전문가에게 연락하십시오.'

In [5]:
from eval import get_eval_data

print(get_eval_data()[36])

P01: 고기랑 술 왕창 먹고 먹튀한 거 봤어?
P02: 먹었으면 돈을 내야지
P02: 그게 뭐야...
P03: 왜 그러는거야 대체?
P03: 못된 심보군 키키
P01: 내 말이
P01: 돈을 떠나서 괘씸하다 키키
P02: 대박이네
P02: 요즘 세상에 그런 생각도 하고 키키
P03: 진짜 돈을 많이 줘도 치우기 싫을 듯 키키
P01: 그러게
P01: 자영업자 가뜩이나 힘든데 ㅠㅠ
P02: 그니까 ㅠㅠ
P02: 진짜 나쁜 사람들 많아
P03: 그니까
P03: 숙박업소 사장님은 무슨 죄야 ㅠㅠ
P01: 키키 숙박업소 아니야
P01: 고깃집 사장님이야
P02: 맞아 키키
P02: 너 기사 대충 읽었지~?
P03: 아 진짜? 키키
P03: 나는 펜션 말하는 줄 키키
P01: 노노
P01: 저 작은 가게에서 엄청 나게 먹어 댔네
P02: 아주 그냥 식성이 좋은 사람들이네 키키
P03: 한번 찾아봐야겠네 키키
P03: 듣기만 해도 화난다
P01: 걔네 완전 계획적이고 상습범이래 키키
P02: 그런 걸로 계획을 세우고 그러냐 에이고
P03: 진짜 머리를 거기다 쓰냐
P03: 너무하다
P03: 몇 명이 그런 거야 대체


In [6]:
from eval import get_eval_data

response = client.generate_content(
    contents=get_eval_data()[36],
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "SAFETY",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "HIGH"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 459,
        "total_token_count": 459
      }
    }),
)

## 3. 별도의 Guardrail 라이브러리
- guardrails-ai
- NVIDIA-NeMo
- guidance

In [2]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("Love how you think and attack the problem. Great job!")

try:
    # Test failing response
    guard.validate(
        "Please look carefully. You are a stupid idiot who can't do anything right."
    )
except Exception as e:
    print(e)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hyeonjinho/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Validation failed for field with errors: The following sentences in your response were found to be toxic:

- You are a stupid idiot who can't do anything right.


/Users/hyeonjinho/Desktop/dev/Learned/llm/fastcampus-prompt-engineering-aillm-service/kakaotalk-summarization/.venv/lib/python3.9/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [1]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("안녕하세요!")

try:
    # Test failing response
    guard.validate(
        "바보 멍청이"
    )
except Exception as e:
    print(e)

/Users/hyeonjinho/Desktop/dev/Learned/llm/fastcampus-prompt-engineering-aillm-service/kakaotalk-summarization/.venv/lib/python3.9/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
